In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1K3QJmxvgc0_ZwXawSeV_oQll-6WeibaO", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/01_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Notebook Overview
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_02_notebook_overview.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# 🚀 Why Do We Need Parallelism? Memory & Compute Bottlenecks

*Part 1 of the Vizuara series on 5D Parallelism from Scratch*
*Estimated time: 30 minutes*

In this notebook, you will learn to calculate exactly how much memory any model needs for training — and understand, with concrete numbers, why a single GPU is never enough for modern LLMs. By the end, you will have built an **interactive memory and compute planner** that visualizes the bottlenecks for any model size.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/5d-parallelism-from-scratch/practice/1/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Code Walkthrough: Plot Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_03_plot_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

# Reproducibility
np.random.seed(42)

# Plot styling for clean, publication-quality figures
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'legend.fontsize': 10,
    'figure.dpi': 100,
    'axes.spines.top': False,
    'axes.spines.right': False,
})

print("Setup complete! Let us begin.")

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_04_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

Training modern large language models requires thousands of GPUs working in concert. Llama 3 405B was trained on 16,384 GPUs. DeepSeek-V3 has 671 billion parameters. These numbers sound impressive, but they raise a fundamental question: **why can we not just use one really powerful GPU?**

Before we learn *how* to parallelize training (which is what the rest of this series covers), we need to deeply understand *why* parallelism is necessary. By the end of this notebook, you will be able to:

- Calculate the exact memory footprint for training any model
- Understand why optimizer states dominate the memory budget
- Estimate training time and minimum GPU requirements
- Build an interactive tool that visualizes all of these bottlenecks

Let us start with a preview of what we are building. Here is the kind of output our final memory calculator will produce:

In [ ]:
#@title 🎧 What to Look For: Teaser Viz Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_05_teaser_viz_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 Teaser: preview of our final output — a stacked bar chart memory breakdown
# (We will build this step by step. For now, just admire the target.)

fig, ax = plt.subplots(figsize=(10, 6))

models = ['7B', '13B', '70B', '405B', '671B']
# Memory in GB for each component (we will derive these numbers soon)
weights_gb =     [14,   26,   140,   810,  1342]
gradients_gb =   [14,   26,   140,   810,  1342]
optimizer_gb =   [84,  156,   840,  4860,  8052]
activations_gb = [40,   60,   180,   500,   600]

x = np.arange(len(models))
width = 0.5

We build the stacked bars layer by layer -- each component sits on top of the previous one:

In [ ]:
#@title 🎧 Code Walkthrough: Teaser Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_06_teaser_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 Visualization
bars1 = ax.bar(x, weights_gb, width, label='Weights (fp16)', color='#3B82F6')
bars2 = ax.bar(x, gradients_gb, width, bottom=weights_gb, label='Gradients (fp16)', color='#10B981')
bars3 = ax.bar(x, optimizer_gb, width,
               bottom=np.array(weights_gb) + np.array(gradients_gb),
               label='Optimizer States (fp32)', color='#F59E0B')
bars4 = ax.bar(x, activations_gb, width,
               bottom=np.array(weights_gb) + np.array(gradients_gb) + np.array(optimizer_gb),
               label='Activations (approx)', color='#8B5CF6', alpha=0.7, linestyle='--')

ax.axhline(y=80, color='red', linestyle='--', linewidth=2, label='A100 GPU Memory (80 GB)')
ax.set_xlabel('Model Size')
ax.set_ylabel('Memory (GB)')
ax.set_title('Training Memory Requirements vs. GPU Capacity')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend(loc='upper left')
ax.set_yscale('log')
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_ylim(10, 20000)

plt.tight_layout()
plt.show()
print("By the end of this notebook, you will know exactly where every byte comes from.")

In [ ]:
#@title 🎧 Listen: Restaurant Analogy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_07_restaurant_analogy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition: The Restaurant Analogy

Before we touch any math, let us build intuition with an analogy.

Imagine you run a restaurant that has just received an order for **1,000 plates of biryani** — but you only have **one chef** and **one stove**.

What happens?

Your chef starts cooking. One plate at a time. Each plate takes about 10 minutes. That means it will take roughly **10,000 minutes — about 7 days** — to finish all 1,000 plates. Your customers have long left by then.

Now think about what exactly is going wrong. There are **two** problems:

1. **The chef is too slow** — one person simply cannot cook 1,000 plates fast enough. This is a *compute bottleneck*.
2. **The kitchen is too small** — even if the chef were faster, a single stove can only hold one pot at a time. This is a *memory bottleneck*.

This is exactly the situation we face when training large language models:
- The **chef** is a single GPU
- The **recipe** is the model (weights, gradients, optimizer states)
- The **1,000 plates** is the massive training dataset
- The **kitchen** is the GPU's memory

In [ ]:
#@title 🎧 Code Walkthrough: Analogy Calculation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_08_analogy_calculation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let us make this concrete with a simple calculation

plates = 1000
time_per_plate_minutes = 10

total_minutes = plates * time_per_plate_minutes
total_hours = total_minutes / 60
total_days = total_hours / 24

print(f"One chef, {plates} plates:")
print(f"  Total time: {total_minutes:,} minutes")
print(f"           = {total_hours:,.0f} hours")
print(f"           = {total_days:.1f} days")
print()

# What if we add more chefs?
for num_chefs in [1, 4, 10, 100]:
    days = total_days / num_chefs
    print(f"  {num_chefs:>3} chef(s): {days:>6.1f} days")

In [ ]:
#@title 🎧 Listen: Five Strategies
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_09_five_strategies.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### The Five Strategies

So how do we scale this kitchen? There are five fundamental strategies, each solving a different aspect of the problem:

| Strategy | Restaurant Analogy | GPU Parallelism |
|---|---|---|
| **Hire more chefs** | Each chef gets the full recipe, splits orders | **Data Parallelism** |
| **Split the recipe** | Tear the recipe in half, each chef handles part | **Tensor Parallelism** |
| **Assembly line** | One chef preps, another cooks, another plates | **Pipeline Parallelism** |
| **Split the orders** | Each chef handles different portions of a long order | **Sequence Parallelism** |
| **Specialist chefs** | One for appetizers, one for mains, one for desserts | **Expert Parallelism** |

These five strategies are exactly what companies like Meta, Google, and DeepSeek use to train models across thousands of GPUs. In the rest of this notebook series, we will implement each one from scratch. But first, we need to understand the *why* — and that means understanding memory.

In [ ]:
#@title 🎧 Narration: Reflection Analogy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_10_reflection_analogy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🤔 Think About This

Before we dive into the math, pause and think:

1. If you had **4 chefs** and could split work perfectly, how long would 1,000 plates take?
2. But what if the **recipe itself** is so complex that no single chef can remember it all? Adding more chefs does not help with that problem.
3. Which of the five strategies addresses the "recipe too complex" problem?

*Take a moment to think, then continue.*

In [ ]:
#@title 🎧 What to Look For: Bottlenecks Viz Compute
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_11_bottlenecks_viz_compute.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 Let us visualize the two bottlenecks side by side

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left: Compute bottleneck — time vs number of chefs
num_chefs = np.array([1, 2, 4, 8, 16, 32, 64, 128])
ideal_days = total_days / num_chefs

ax1.plot(num_chefs, ideal_days, 'o-', color='#3B82F6', linewidth=2, markersize=8)
ax1.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='1 day target')
ax1.set_xlabel('Number of Chefs (GPUs)')
ax1.set_ylabel('Time (days)')
ax1.set_title('Compute Bottleneck\n"The chef is too slow"')
ax1.set_xscale('log', base=2)
ax1.set_yscale('log')
ax1.legend()
ax1.grid(True, alpha=0.3)

In [ ]:
#@title 🎧 Listen: Bottlenecks Viz Memory Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_12_bottlenecks_viz_memory_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now the memory bottleneck -- model size vs. available GPU memory:

In [ ]:
#@title 🎧 What to Look For: Bottlenecks Viz Memory Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_13_bottlenecks_viz_memory_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 Right: Memory bottleneck — model size vs GPU memory
model_sizes_B = np.array([1, 3, 7, 13, 30, 70, 175, 405, 671])
memory_needed_GB = model_sizes_B * 16  # 16 bytes per parameter (we will derive this)
gpu_memory = 80  # A100

colors = ['#10B981' if m <= gpu_memory else '#EF4444' for m in memory_needed_GB]
ax2.bar(range(len(model_sizes_B)), memory_needed_GB, color=colors, edgecolor='white')
ax2.axhline(y=gpu_memory, color='red', linestyle='--', linewidth=2, label=f'A100 Memory ({gpu_memory} GB)')
ax2.set_xlabel('Model Size')
ax2.set_ylabel('Training Memory (GB)')
ax2.set_title('Memory Bottleneck\n"The kitchen is too small"')
ax2.set_xticks(range(len(model_sizes_B)))
ax2.set_xticklabels([f'{s}B' for s in model_sizes_B], rotation=45)
ax2.legend()

plt.tight_layout()
plt.show()

print("Green = fits on one A100, Red = does not fit.")
print("Notice: even a 7B model does not fit!")

In [ ]:
#@title 🎧 Listen: Math Of Memory
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_14_math_of_memory.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics of Memory

Now let us formalize exactly where all this memory goes. When we train a model using **mixed-precision training** with the **Adam optimizer** (which is what virtually all modern LLMs use), each parameter requires memory for four things.

### 3.1 The Four Memory Components

**Component 1: Model Weights (fp16) — 2 bytes per parameter**

The model weights are stored in 16-bit floating point (fp16 or bfloat16) during the forward and backward pass. Each fp16 value takes 2 bytes.

**Component 2: Gradients (fp16) — 2 bytes per parameter**

During backpropagation, we compute the gradient of the loss with respect to each parameter. These are also stored in fp16, so 2 bytes per parameter.

**Component 3: Optimizer States (fp32) — 12 bytes per parameter**

This is where Adam gets expensive. Adam maintains:
- A **fp32 copy** of the weights: 4 bytes per parameter
- The **first moment** (moving average of gradients): 4 bytes per parameter
- The **second moment** (moving average of squared gradients): 4 bytes per parameter

That is $4 + 4 + 4 = 12$ bytes per parameter — just for the optimizer.

### The Total

Putting it all together, the memory for model state (excluding activations) is:

$$M_{\text{model}} = P \times (2 + 2 + 12) = 16P \text{ bytes}$$

where $P$ is the number of parameters.

Computationally, this equation says: for every single parameter in your model, you need to store the parameter itself (2 bytes), its gradient (2 bytes), and three optimizer buffers (12 bytes). The optimizer alone accounts for **75%** of the model state memory.

In [ ]:
#@title 🎧 Code Walkthrough: Memory Breakdown Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_15_memory_breakdown_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def memory_breakdown_bytes(num_params):
    """
    Calculate memory breakdown for mixed-precision training with Adam.

    Args:
        num_params: Number of model parameters

    Returns:
        Dictionary with memory in bytes for each component
    """
    weights_bytes = num_params * 2          # fp16 weights
    gradients_bytes = num_params * 2        # fp16 gradients
    optimizer_fp32_copy = num_params * 4    # fp32 master weights
    optimizer_first_moment = num_params * 4 # Adam m (first moment)
    optimizer_second_moment = num_params * 4 # Adam v (second moment)

    optimizer_total = optimizer_fp32_copy + optimizer_first_moment + optimizer_second_moment

    return {
        'weights': weights_bytes,
        'gradients': gradients_bytes,
        'optimizer_fp32_copy': optimizer_fp32_copy,
        'optimizer_first_moment': optimizer_first_moment,
        'optimizer_second_moment': optimizer_second_moment,
        'optimizer_total': optimizer_total,
        'total': weights_bytes + gradients_bytes + optimizer_total,
    }

In [ ]:
#@title 🎧 Code Walkthrough: Bytes To Gb Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_16_bytes_to_gb_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


A small helper to convert raw byte counts to human-readable gigabytes:

In [ ]:
def bytes_to_gb(b):
    """Convert bytes to gigabytes."""
    return b / (1024 ** 3)

In [ ]:
#@title 🎧 Code Walkthrough: 7b Model Breakdown
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_17_7b_model_breakdown.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us apply this to a 7B parameter model and see the breakdown:

In [ ]:
# Compute for a 7B model
P = 7e9
breakdown = memory_breakdown_bytes(P)

print(f"Memory breakdown for a {P/1e9:.0f}B parameter model:")
print(f"  Weights (fp16):           {bytes_to_gb(breakdown['weights']):>8.1f} GB")
print(f"  Gradients (fp16):         {bytes_to_gb(breakdown['gradients']):>8.1f} GB")
print(f"  Optimizer fp32 copy:      {bytes_to_gb(breakdown['optimizer_fp32_copy']):>8.1f} GB")
print(f"  Optimizer first moment:   {bytes_to_gb(breakdown['optimizer_first_moment']):>8.1f} GB")
print(f"  Optimizer second moment:  {bytes_to_gb(breakdown['optimizer_second_moment']):>8.1f} GB")
print(f"  ─────────────────────────────────────")
print(f"  Optimizer total:          {bytes_to_gb(breakdown['optimizer_total']):>8.1f} GB")
print(f"  ─────────────────────────────────────")
print(f"  TOTAL model state:        {bytes_to_gb(breakdown['total']):>8.1f} GB")
print()
print(f"A100 GPU memory: 80 GB")
print(f"Deficit: {bytes_to_gb(breakdown['total']) - 80:.1f} GB over budget!")

In [ ]:
#@title 🎧 Listen: Optimizer Insight
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_18_optimizer_insight.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 💡 Key Insight: Optimizer States Dominate

Notice something striking: the optimizer states (78 GB) account for **75%** of the total memory. The actual model weights are only 13 GB. This is a crucial insight — when we learn about ZeRO (Zero Redundancy Optimizer) in the Data Parallelism notebook, we will see that the first thing it does is shard the optimizer states across GPUs, because that gives the biggest bang for the buck.

In [ ]:
#@title 🎧 What to Look For: 7b Viz Pie Chart
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_19_7b_viz_pie_chart.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 ation Checkpoint 1: Pie chart of memory components

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart showing the breakdown
labels = ['Weights\n(fp16)', 'Gradients\n(fp16)', 'Optimizer\nfp32 copy',
          'Optimizer\n1st moment', 'Optimizer\n2nd moment']
sizes = [
    bytes_to_gb(breakdown['weights']),
    bytes_to_gb(breakdown['gradients']),
    bytes_to_gb(breakdown['optimizer_fp32_copy']),
    bytes_to_gb(breakdown['optimizer_first_moment']),
    bytes_to_gb(breakdown['optimizer_second_moment']),
]
colors = ['#3B82F6', '#10B981', '#F59E0B', '#EF4444', '#8B5CF6']
explode = (0, 0, 0.05, 0.05, 0.05)  # Emphasize optimizer components

wedges, texts, autotexts = ax1.pie(
    sizes, labels=labels, autopct='%1.0f%%', colors=colors,
    explode=explode, startangle=90, textprops={'fontsize': 10}
)
ax1.set_title(f'Memory Breakdown: {P/1e9:.0f}B Model\n(Total: {bytes_to_gb(breakdown["total"]):.0f} GB)')

In [ ]:
#@title 🎧 What to Look For: 7b Viz Stacked Bar
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_20_7b_viz_stacked_bar.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now we add a stacked bar comparing the 7B model's memory against GPU capacity:

In [ ]:
# 📊 Stacked bar for 7B model vs GPU capacity
categories = ['Weights', 'Gradients', 'Optimizer']
values = [
    bytes_to_gb(breakdown['weights']),
    bytes_to_gb(breakdown['gradients']),
    bytes_to_gb(breakdown['optimizer_total']),
]
bar_colors = ['#3B82F6', '#10B981', '#F59E0B']

bottom = 0
for cat, val, col in zip(categories, values, bar_colors):
    ax2.bar('7B Model', val, bottom=bottom, color=col, label=f'{cat}: {val:.0f} GB',
            edgecolor='white', linewidth=0.5)
    bottom += val

ax2.axhline(y=80, color='red', linestyle='--', linewidth=2, label='A100 (80 GB)')
ax2.set_ylabel('Memory (GB)')
ax2.set_title('7B Model: Does It Fit on One A100?')
ax2.legend(loc='upper right')
ax2.set_ylim(0, 130)
ax2.annotate(
    f'Exceeds A100 by\n{bytes_to_gb(breakdown["total"]) - 80:.0f} GB!',
    xy=(0, 80), xytext=(0.5, 60),
    fontsize=11, color='red', fontweight='bold',
    arrowprops=dict(arrowstyle='->', color='red', lw=2),
    ha='center'
)

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Activations Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_21_activations_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.2 But Wait — What About Activations?

The memory breakdown above covers the *model state*: weights, gradients, and optimizer states. But there is another major memory consumer: **activations**.

During the forward pass, every layer produces intermediate outputs. These intermediate tensors must be **saved in memory** because the backward pass needs them to compute gradients. This is a fundamental property of backpropagation — you cannot compute gradients without the activations from the forward pass.

The activation memory depends on the model architecture and the training configuration:

$$M_{\text{act}} \approx 2 \times B \times S \times H \times L \times b_{\text{elem}}$$

where:
- $B$ = batch size (number of sequences processed at once)
- $S$ = sequence length (number of tokens per sequence)
- $H$ = hidden dimension (width of each layer)
- $L$ = number of layers (depth of the model)
- $b_{\text{elem}}$ = bytes per element (2 for fp16)
- The factor of 2 accounts for both the layer input and the attention intermediate states (a rough approximation)

Computationally, this says: for every token in every sequence in the batch, at every layer, we need to store the hidden representation. The factor of 2 is a simplification — real Transformers store additional intermediates (attention scores, post-LayerNorm values, etc.), but this gives the right order of magnitude.

> **Warning**: This is a simplified formula. The real activation memory includes attention score matrices (which scale as $B \times S^2$ per head per layer), post-normalization values, MLP intermediates, and more. For a full treatment, see the Megatron-LM paper. Our formula captures the dominant term for typical configurations.

In [ ]:
#@title 🎧 Code Walkthrough: Activation Memory Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_22_activation_memory_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def activation_memory_gb(batch_size, seq_len, hidden_dim, num_layers,
                          bytes_per_element=2):
    """
    Estimate activation memory for a Transformer model.
    Includes hidden states, attention scores, and MLP intermediates.

    Args:
        batch_size: Number of sequences per batch
        seq_len: Tokens per sequence
        hidden_dim: Hidden dimension (model width)
        num_layers: Number of Transformer layers
        bytes_per_element: Bytes per value (2 for fp16/bf16)

    Returns:
        Estimated activation memory in GB
    """
    # Main hidden state activations: input to each layer
    hidden_state_bytes = batch_size * seq_len * hidden_dim * num_layers * bytes_per_element

    # Attention intermediates: Q, K, V projections + attention scores
    num_heads = max(hidden_dim // 128, 1)  # Typical head dim = 128
    attn_scores_bytes = batch_size * num_heads * seq_len * seq_len * bytes_per_element * num_layers

    # MLP intermediates: typically 4x expansion
    mlp_bytes = batch_size * seq_len * (4 * hidden_dim) * num_layers * bytes_per_element

    total_bytes = hidden_state_bytes + attn_scores_bytes + mlp_bytes
    return total_bytes / (1024 ** 3)

In [ ]:
#@title 🎧 Code Walkthrough: Activation Scaling
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_23_activation_scaling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us apply this function to a 7B model and see how activation memory scales with batch size:

In [ ]:
# Typical architecture for a 7B model (similar to Llama 2 7B)
act_mem = activation_memory_gb(
    batch_size=1,
    seq_len=2048,
    hidden_dim=4096,
    num_layers=32
)

print(f"Activation memory estimate for 7B model (batch_size=1, seq_len=2048):")
print(f"  {act_mem:.1f} GB")
print()

# Show how it grows with batch size
print("How activation memory scales with batch size:")
for bs in [1, 2, 4, 8, 16]:
    mem = activation_memory_gb(bs, 2048, 4096, 32)
    print(f"  batch_size={bs:>2}: {mem:>7.1f} GB")

In [ ]:
#@title 🎧 What to Look For: Activation Viz Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_24_activation_viz_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us visualize how activation memory grows with batch size and sequence length:

In [ ]:
#@title 🎧 Code Walkthrough: Activation Viz Batch Size
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_25_activation_viz_batch_size.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 ation Checkpoint 2: Activation memory growth

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Left: activation memory vs batch size
batch_sizes = np.arange(1, 33)
act_mems = [activation_memory_gb(bs, 2048, 4096, 32) for bs in batch_sizes]

ax1.plot(batch_sizes, act_mems, 'o-', color='#8B5CF6', linewidth=2, markersize=4)
ax1.axhline(y=80, color='red', linestyle='--', linewidth=1.5, label='A100 (80 GB)')
ax1.fill_between(batch_sizes, act_mems, alpha=0.2, color='#8B5CF6')
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('Activation Memory (GB)')
ax1.set_title('Activation Memory vs. Batch Size\n(7B model, seq_len=2048)')
ax1.legend()
ax1.grid(True, alpha=0.3)

In [ ]:
#@title 🎧 Listen: Activation Viz Seq Len Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_26_activation_viz_seq_len_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Sequence length has an even more dramatic effect because attention scores scale quadratically:

In [ ]:
#@title 🎧 What to Look For: Activation Viz Seq Len Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_27_activation_viz_seq_len_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 Right: activation memory vs sequence length
seq_lens = [512, 1024, 2048, 4096, 8192, 16384, 32768]
act_mems_seq = [activation_memory_gb(1, sl, 4096, 32) for sl in seq_lens]

ax2.bar(range(len(seq_lens)), act_mems_seq, color='#8B5CF6', alpha=0.8, edgecolor='white')
ax2.axhline(y=80, color='red', linestyle='--', linewidth=1.5, label='A100 (80 GB)')
ax2.set_xlabel('Sequence Length')
ax2.set_ylabel('Activation Memory (GB)')
ax2.set_title('Activation Memory vs. Sequence Length\n(7B model, batch_size=1)')
ax2.set_xticks(range(len(seq_lens)))
ax2.set_xticklabels([f'{sl//1024}K' if sl >= 1024 else str(sl) for sl in seq_lens], rotation=45)
ax2.legend()

plt.tight_layout()
plt.show()

print("Notice: activation memory grows linearly with batch size,")
print("but can grow SUPER-linearly with sequence length (due to attention scores).")

In [ ]:
#@title 🎧 Transition: Build Calculator Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_28_build_calculator_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Build It — Component by Component

### 4.1 A Complete Memory Calculator

Now let us put everything together into one clean function that computes the full memory breakdown.

In [ ]:
#@title 🎧 Code Walkthrough: Full Memory Calculator Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_29_full_memory_calculator_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def compute_full_memory(
    num_params_billions, batch_size=1, seq_len=2048,
    hidden_dim=4096, num_layers=32,
    weight_bytes=2, grad_bytes=2,
    optimizer_bytes=12, activation_bytes_per_elem=2,
):
    """Compute the full training memory breakdown for a model.
    Returns dict with weights_gb, gradients_gb, optimizer_gb,
    activations_gb, total_gb, and num_params_B."""
    P = num_params_billions * 1e9

    weights_gb = (P * weight_bytes) / (1024**3)
    gradients_gb = (P * grad_bytes) / (1024**3)
    optimizer_gb = (P * optimizer_bytes) / (1024**3)
    activations_gb = activation_memory_gb(
        batch_size, seq_len, hidden_dim, num_layers, activation_bytes_per_elem
    )
    total_gb = weights_gb + gradients_gb + optimizer_gb + activations_gb

    return {'weights_gb': weights_gb, 'gradients_gb': gradients_gb,
            'optimizer_gb': optimizer_gb, 'activations_gb': activations_gb,
            'total_gb': total_gb, 'num_params_B': num_params_billions}

In [ ]:
#@title 🎧 Code Walkthrough: Test Full Calculator
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_30_test_full_calculator.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us test this function with our 7B model and verify the numbers match our earlier calculation:

In [ ]:
# Test with our 7B model
result = compute_full_memory(7, batch_size=1, seq_len=2048, hidden_dim=4096, num_layers=32)

print(f"Full memory breakdown for 7B model:")
print(f"  Weights:       {result['weights_gb']:>8.1f} GB  ({result['weights_gb']/result['total_gb']*100:.0f}%)")
print(f"  Gradients:     {result['gradients_gb']:>8.1f} GB  ({result['gradients_gb']/result['total_gb']*100:.0f}%)")
print(f"  Optimizer:     {result['optimizer_gb']:>8.1f} GB  ({result['optimizer_gb']/result['total_gb']*100:.0f}%)")
print(f"  Activations:   {result['activations_gb']:>8.1f} GB  ({result['activations_gb']/result['total_gb']*100:.0f}%)")
print(f"  ──────────────────────────────")
print(f"  TOTAL:         {result['total_gb']:>8.1f} GB")
print(f"\n  A100 memory:       80.0 GB")
print(f"  Overflow:      {result['total_gb'] - 80:.1f} GB")

In [ ]:
#@title 🎧 Listen: Scaling To Real Models Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_31_scaling_to_real_models_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Scaling to Real Models

Now let us compute memory requirements for the models that are actually being trained today. We will use approximate architecture parameters for each.

In [ ]:
#@title 🎧 Code Walkthrough: Real Model Configs Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_32_real_model_configs_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Real model configurations (approximate)
model_configs = {
    '7B':   {'params': 7,   'hidden': 4096,  'layers': 32,  'heads': 32},
    '13B':  {'params': 13,  'hidden': 5120,  'layers': 40,  'heads': 40},
    '70B':  {'params': 70,  'hidden': 8192,  'layers': 80,  'heads': 64},
    '405B': {'params': 405, 'hidden': 16384, 'layers': 126, 'heads': 128},
    '671B': {'params': 671, 'hidden': 7168,  'layers': 61,  'heads': 128},  # DeepSeek-V3 (MoE)
}

print(f"{'Model':<8} {'Weights':>10} {'Gradients':>10} {'Optimizer':>10} {'Activations':>12} {'TOTAL':>10} {'Fits A100?':>12}")
print("─" * 85)

results = {}
for name, cfg in model_configs.items():
    r = compute_full_memory(
        cfg['params'],
        batch_size=1,
        seq_len=2048,
        hidden_dim=cfg['hidden'],
        num_layers=cfg['layers'],
    )
    results[name] = r
    fits = "Yes" if r['total_gb'] <= 80 else "No"
    emoji = "  ✅" if fits == "Yes" else "  ❌"
    print(f"{name:<8} {r['weights_gb']:>8.1f}GB {r['gradients_gb']:>8.1f}GB "
          f"{r['optimizer_gb']:>8.1f}GB {r['activations_gb']:>10.1f}GB "
          f"{r['total_gb']:>8.1f}GB {emoji}")

In [ ]:
#@title 🎧 What to Look For: Visualize All Models Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_33_visualize_all_models_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us visualize all five models in a single stacked bar chart to see the full picture:

In [ ]:
# 📊 ation Checkpoint 3: Stacked bar chart comparing all models

fig, ax = plt.subplots(figsize=(12, 7))

model_names = list(results.keys())
x = np.arange(len(model_names))
width = 0.55

weights = [results[m]['weights_gb'] for m in model_names]
grads = [results[m]['gradients_gb'] for m in model_names]
optim = [results[m]['optimizer_gb'] for m in model_names]
acts = [results[m]['activations_gb'] for m in model_names]

In [ ]:
#@title 🎧 Code Walkthrough: Visualize All Models Drawing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_34_visualize_all_models_drawing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


We draw each memory component as a stacked layer and add reference lines for GPU capacity:

In [ ]:
b1 = ax.bar(x, weights, width, label='Weights (fp16)', color='#3B82F6', edgecolor='white')
b2 = ax.bar(x, grads, width, bottom=weights, label='Gradients (fp16)', color='#10B981', edgecolor='white')
b3 = ax.bar(x, optim, width,
            bottom=np.array(weights) + np.array(grads),
            label='Optimizer States (fp32)', color='#F59E0B', edgecolor='white')
b4 = ax.bar(x, acts, width,
            bottom=np.array(weights) + np.array(grads) + np.array(optim),
            label='Activations (est.)', color='#8B5CF6', alpha=0.75, edgecolor='white')

# GPU memory lines
ax.axhline(y=80, color='#EF4444', linestyle='--', linewidth=2.5, label='A100 80GB', zorder=5)
ax.axhline(y=80, color='#06B6D4', linestyle='-.', linewidth=2, label='H100 80GB', zorder=5)

ax.set_xlabel('Model', fontsize=13)
ax.set_ylabel('Training Memory (GB)', fontsize=13)
ax.set_title('Training Memory Requirements Across Model Sizes\n(Mixed-precision, Adam optimizer, batch_size=1, seq_len=2048)', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(model_names, fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.set_yscale('log')
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_ylim(10, 20000)
ax.grid(axis='y', alpha=0.3)

In [ ]:
#@title 🎧 What to Look For: Visualize All Models Render
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_35_visualize_all_models_render.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Finally, annotate each bar with the total memory and render the chart:

In [ ]:
# 📊 Annotate totals
for i, m in enumerate(model_names):
    total = results[m]['total_gb']
    ax.text(i, total * 1.15, f'{total:.0f} GB', ha='center', va='bottom',
            fontsize=10, fontweight='bold', color='#1F2937')

plt.tight_layout()
plt.show()

print("Every model from 7B and above exceeds a single A100.")
print("The 405B model needs over 50x the memory of one A100!")

In [ ]:
#@title 🎧 Before You Start: Todo Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_36_todo_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. 🔧 Your Turn

Now it is your turn. Complete the following exercises to solidify your understanding.

### 🔧 TODO 1: Implement the Full Memory Calculator

Complete the function below that takes a model size and returns the **total** memory required, including both model state and activations. We have provided the skeleton — fill in the blanks.

In [ ]:
#@title 🎧 Before You Start: Todo Memory Calculator
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_37_todo_1_memory_calculator.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def total_training_memory_gb(num_params_billions, batch_size, seq_len,
                              hidden_dim, num_layers):
    """
    Calculate total training memory in GB.
    Returns the sum of weights, gradients, optimizer states, and activations.
    """
    P = num_params_billions * 1e9

    # ============ TODO ============
    # Step 1: Calculate weight memory in GB (fp16 = 2 bytes per param)
    # Step 2: Calculate gradient memory in GB (fp16 = 2 bytes per param)
    # Step 3: Calculate optimizer memory in GB (Adam fp32 = 12 bytes per param)
    # Step 4: Calculate activation memory using activation_memory_gb()
    # Step 5: Sum all four components
    # ==============================

    weights_gb = ???      # YOUR CODE HERE
    gradients_gb = ???    # YOUR CODE HERE
    optimizer_gb = ???    # YOUR CODE HERE
    act_gb = ???          # YOUR CODE HERE

    total_gb = ???        # YOUR CODE HERE

    return total_gb

In [ ]:
#@title 🎧 Code Walkthrough: Todo Verification
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_38_todo_1_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Todo Verification
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_40_todo_2_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Verification: Run this cell to check your implementation

# Test case 1: 7B model
expected_7b = compute_full_memory(7, batch_size=1, seq_len=2048,
                                   hidden_dim=4096, num_layers=32)['total_gb']
your_7b = total_training_memory_gb(7, batch_size=1, seq_len=2048,
                                    hidden_dim=4096, num_layers=32)

assert abs(your_7b - expected_7b) < 0.1, \
    f"For 7B model: expected {expected_7b:.1f} GB, got {your_7b:.1f} GB"

# Test case 2: 70B model
expected_70b = compute_full_memory(70, batch_size=2, seq_len=4096,
                                    hidden_dim=8192, num_layers=80)['total_gb']
your_70b = total_training_memory_gb(70, batch_size=2, seq_len=4096,
                                     hidden_dim=8192, num_layers=80)

assert abs(your_70b - expected_70b) < 0.1, \
    f"For 70B model: expected {expected_70b:.1f} GB, got {your_70b:.1f} GB"

print("Correct! Your total_training_memory_gb function works perfectly.")
print(f"   7B model:  {your_7b:.1f} GB")
print(f"   70B model: {your_70b:.1f} GB")

In [ ]:
#@title 🎧 Before You Start: Todo Min Gpus
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_39_todo_2_min_gpus.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🔧 TODO 2: Minimum Number of GPUs

Given a model size and GPU memory capacity, compute the **minimum number of GPUs** needed if we could perfectly shard everything (weights, gradients, optimizer states, and activations) across GPUs. In reality, there is overhead from communication and imperfect sharding, but this gives a theoretical lower bound.

In [ ]:
def min_gpus_needed(num_params_billions, gpu_memory_gb=80,
                    batch_size=1, seq_len=2048,
                    hidden_dim=4096, num_layers=32):
    """
    Compute the minimum number of GPUs needed to train a model,
    assuming perfect memory sharding.

    Args:
        num_params_billions: Parameters in billions
        gpu_memory_gb: Memory per GPU in GB
        batch_size, seq_len, hidden_dim, num_layers: Model/training config

    Returns:
        min_gpus (int): Minimum number of GPUs (ceiling division)
    """
    # ============ TODO ============
    # Step 1: Calculate total memory using compute_full_memory()
    # Step 2: Divide by GPU memory and round UP (use int() + 1 trick or math.ceil)
    # ==============================

    total_mem = ???       # YOUR CODE HERE
    min_gpus = ???        # YOUR CODE HERE

    return min_gpus

In [ ]:
# Verification: Run this cell to check your implementation
import math

# 7B model on A100 (80 GB)
result_7b = compute_full_memory(7, batch_size=1, seq_len=2048, hidden_dim=4096, num_layers=32)
expected_gpus_7b = math.ceil(result_7b['total_gb'] / 80)
your_gpus_7b = min_gpus_needed(7, gpu_memory_gb=80, batch_size=1, seq_len=2048,
                                hidden_dim=4096, num_layers=32)

assert your_gpus_7b == expected_gpus_7b, \
    f"For 7B model: expected {expected_gpus_7b} GPUs, got {your_gpus_7b}"

# 405B model on A100 (80 GB)
result_405b = compute_full_memory(405, batch_size=1, seq_len=2048, hidden_dim=16384, num_layers=126)
expected_gpus_405b = math.ceil(result_405b['total_gb'] / 80)
your_gpus_405b = min_gpus_needed(405, gpu_memory_gb=80, batch_size=1, seq_len=2048,
                                  hidden_dim=16384, num_layers=126)

assert your_gpus_405b == expected_gpus_405b, \
    f"For 405B model: expected {expected_gpus_405b} GPUs, got {your_gpus_405b}"

print("Correct! Your min_gpus_needed function works perfectly.")
print(f"   7B model on A100:   {your_gpus_7b} GPUs minimum")
print(f"   405B model on A100: {your_gpus_405b} GPUs minimum")
print(f"\n   In practice, you need many more GPUs than this due to")
print(f"   communication overhead and imperfect sharding.")

In [ ]:
#@title 🎧 Listen: Compute Bottleneck Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_41_compute_bottleneck_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. The Compute Bottleneck

So far we have focused on *memory*. But even if memory were not a problem — imagine GPUs with infinite memory — training would still take an enormous amount of time on a single GPU. This is the **compute bottleneck**.

### 6.1 Estimating Training FLOPs

A widely used rule of thumb (from the Chinchilla paper and others) estimates the total floating-point operations for training as:

$$\text{FLOPs}_{\text{train}} \approx 6 \times P \times D$$

where:
- $P$ = number of model parameters
- $D$ = number of training tokens (dataset size)
- The factor of 6 comes from: 2 FLOPs per parameter per token for the forward pass (multiply-add), and approximately 2x that for the backward pass, giving $2 \times 3 = 6$.

Computationally, this says: for each token in the dataset, each parameter participates in a multiply-add during the forward pass, and the backward pass costs roughly twice that. So the total work scales with the product of parameters and tokens.

In [ ]:
#@title 🎧 Code Walkthrough: Estimate Flops Func
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_42_estimate_flops_func.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def estimate_training_flops(num_params_billions, num_tokens_trillions):
    """
    Estimate total FLOPs for training using the 6PD rule.

    Args:
        num_params_billions: Model parameters in billions
        num_tokens_trillions: Training tokens in trillions

    Returns:
        Total FLOPs as a float
    """
    P = num_params_billions * 1e9
    D = num_tokens_trillions * 1e12
    return 6 * P * D

In [ ]:
#@title 🎧 Code Walkthrough: Flops To Gpu Hours Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_43_flops_to_gpu_hours_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now we need a way to convert raw FLOPs into wall-clock GPU-hours:

In [ ]:
def flops_to_gpu_hours(total_flops, gpu_tflops=312):
    """
    Convert total FLOPs to GPU-hours.

    Args:
        total_flops: Total training FLOPs
        gpu_tflops: GPU throughput in TFLOPS (A100 fp16 ~312 TFLOPS peak)

    Returns:
        GPU-hours (assuming ~50% utilization, which is typical)
    """
    utilization = 0.50  # Typical MFU (Model FLOPS Utilization)
    effective_tflops = gpu_tflops * utilization
    effective_flops_per_second = effective_tflops * 1e12
    seconds = total_flops / effective_flops_per_second
    hours = seconds / 3600
    return hours

In [ ]:
#@title 🎧 Code Walkthrough: Llama3 Compute Estimate
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_44_llama3_compute_estimate.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us apply these functions to estimate how long it would take to train Llama 3 405B:

In [ ]:
# Llama 3 405B: 405B params, ~15T tokens
flops_405b = estimate_training_flops(405, 15)
gpu_hours_405b = flops_to_gpu_hours(flops_405b, gpu_tflops=312)

print(f"Llama 3 405B Training Estimate:")
print(f"  Parameters:     405B")
print(f"  Training tokens: 15T")
print(f"  Total FLOPs:     {flops_405b:.2e}")
print(f"  A100 GPU-hours:  {gpu_hours_405b:,.0f}")
print(f"  On 1 GPU:        {gpu_hours_405b / (24*365):.0f} years")
print(f"  On 16,384 GPUs:  {gpu_hours_405b / 16384 / 24:.0f} days")

In [ ]:
#@title 🎧 Code Walkthrough: Real Training Runs Table
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_45_real_training_runs_table.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let us compute this for several real training runs

training_runs = [
    {'name': 'Llama 2 7B',      'params': 7,   'tokens': 2,    'gpus_used': 1024},
    {'name': 'Llama 2 70B',     'params': 70,  'tokens': 2,    'gpus_used': 2048},
    {'name': 'Llama 3 8B',      'params': 8,   'tokens': 15,   'gpus_used': 2048},
    {'name': 'Llama 3 70B',     'params': 70,  'tokens': 15,   'gpus_used': 8192},
    {'name': 'Llama 3 405B',    'params': 405, 'tokens': 15,   'gpus_used': 16384},
    {'name': 'DeepSeek-V3',     'params': 671, 'tokens': 14.8, 'gpus_used': 2048},
]

print(f"{'Model':<18} {'FLOPs':>12} {'1 GPU (yrs)':>12} {'Actual GPUs':>12} {'Est. Days':>10}")
print("─" * 70)

for run in training_runs:
    flops = estimate_training_flops(run['params'], run['tokens'])
    gpu_hours = flops_to_gpu_hours(flops)
    years_1gpu = gpu_hours / (24 * 365)
    days_actual = gpu_hours / run['gpus_used'] / 24

    print(f"{run['name']:<18} {flops:>10.1e}  {years_1gpu:>10.0f}  {run['gpus_used']:>10,}  {days_actual:>8.0f}")

print()
print("DeepSeek-V3 uses MoE, so only ~37B params are active per token.")
print("   The actual FLOPs are much lower than the total parameter count suggests.")

In [ ]:
#@title 🎧 What to Look For: Training Time Viz Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_46_training_time_viz_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us visualize how training time drops as we add more GPUs. We set up the data first:

In [ ]:
#@title 🎧 What to Look For: Training Time Viz Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_47_training_time_viz_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 📊 ation: Training time on different GPU counts

fig, ax = plt.subplots(figsize=(12, 6))

# Focus on Llama 3 405B as our example
flops_405b = estimate_training_flops(405, 15)
gpu_counts = [1, 8, 64, 512, 2048, 8192, 16384, 32768]

training_days = []
for n_gpus in gpu_counts:
    gpu_hours = flops_to_gpu_hours(flops_405b)
    days = gpu_hours / n_gpus / 24
    training_days.append(days)

colors = ['#EF4444' if d > 365 else '#F59E0B' if d > 30 else '#10B981' for d in training_days]
bars = ax.bar(range(len(gpu_counts)), training_days, color=colors, edgecolor='white')

ax.set_xlabel('Number of A100 GPUs', fontsize=12)
ax.set_ylabel('Training Time (days)', fontsize=12)
ax.set_title('Llama 3 405B: Training Time vs. GPU Count\n(15T tokens, 50% MFU)', fontsize=14)
ax.set_xticks(range(len(gpu_counts)))
ax.set_xticklabels([f'{g:,}' for g in gpu_counts], rotation=45)
ax.set_yscale('log')
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())

Add value labels and reference lines to the chart:

In [ ]:
# 📊 Add value labels on bars
for i, (bar, days) in enumerate(zip(bars, training_days)):
    if days > 365:
        label = f'{days/365:.0f} yrs'
    elif days > 1:
        label = f'{days:.0f} days'
    else:
        label = f'{days*24:.0f} hrs'
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() * 1.2,
            label, ha='center', va='bottom', fontsize=9, fontweight='bold')

# Reference lines
ax.axhline(y=90, color='gray', linestyle=':', alpha=0.5)
ax.text(len(gpu_counts) - 0.5, 90, 'Meta target (~90 days)', ha='right', va='bottom',
        fontsize=9, color='gray')

plt.tight_layout()
plt.show()

print("Red = years, Yellow = months, Green = weeks or less.")
print(f"At 16,384 GPUs (what Meta used), training takes ~{training_days[6]:.0f} days.")

In [ ]:
#@title 🎧 Narration: Final Planner Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_48_final_planner_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Final Output: Interactive Memory & Compute Planner

Now let us bring everything together into a single interactive tool. This is our culminating output — a comprehensive planner that takes any model configuration and produces a full visual report.

We build the planner in stages. First, the function signature and memory/compute calculations:

In [ ]:
#@title 🎧 Code Walkthrough: Planner Function Calc
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_49_planner_function_calc.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def memory_compute_planner(
    num_params_billions,
    batch_size=1,
    seq_len=2048,
    hidden_dim=4096,
    num_layers=32,
    num_tokens_trillions=1.0,
    gpu_name='A100',
    gpu_memory_gb=80,
    gpu_tflops=312,
):
    """
    Complete memory and compute planner for LLM training.

    Produces a comprehensive visual report showing:
    - Stacked bar chart of memory breakdown
    - Comparison against GPU capacity
    - Compute time estimates
    - Minimum GPU requirements
    """
    # -- Compute memory breakdown --
    mem = compute_full_memory(
        num_params_billions, batch_size, seq_len, hidden_dim, num_layers
    )

    # -- Compute training FLOPs and time --
    total_flops = estimate_training_flops(num_params_billions, num_tokens_trillions)
    gpu_hours_1 = flops_to_gpu_hours(total_flops, gpu_tflops)
    min_gpus = math.ceil(mem['total_gb'] / gpu_memory_gb)

In [ ]:
#@title 🎧 Code Walkthrough: Planner Subplot
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_50_planner_subplot_1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Next, we create the figure and draw the memory breakdown stacked bar (subplot 1):

In [ ]:
    # -- Figure with 4 subplots --
    fig = plt.figure(figsize=(16, 12))
    fig.suptitle(
        f"Training Planner: {num_params_billions}B Parameter Model",
        fontsize=18, fontweight='bold', y=0.98
    )

    # Subplot 1: Stacked bar — memory breakdown
    ax1 = fig.add_subplot(2, 2, 1)
    components = ['Weights\n(fp16)', 'Gradients\n(fp16)', 'Optimizer\n(fp32)', 'Activations']
    values = [mem['weights_gb'], mem['gradients_gb'], mem['optimizer_gb'], mem['activations_gb']]
    colors = ['#3B82F6', '#10B981', '#F59E0B', '#8B5CF6']

    bottom = 0
    for comp, val, col in zip(components, values, colors):
        ax1.bar('Memory', val, bottom=bottom, color=col,
                label=f'{comp}: {val:.1f} GB', edgecolor='white', width=0.5)
        bottom += val

    ax1.axhline(y=gpu_memory_gb, color='red', linestyle='--',
                linewidth=2.5, label=f'{gpu_name} ({gpu_memory_gb} GB)')
    ax1.set_ylabel('Memory (GB)')
    ax1.set_title('Memory Breakdown')
    ax1.legend(loc='upper right', fontsize=9)

    if mem['total_gb'] > gpu_memory_gb:
        ax1.annotate(f"Exceeds {gpu_name} by\n{mem['total_gb'] - gpu_memory_gb:.0f} GB",
            xy=(0, gpu_memory_gb), xytext=(0.35, gpu_memory_gb * 0.6),
            fontsize=10, color='red', fontweight='bold',
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

In [ ]:
#@title 🎧 Code Walkthrough: Planner Subplot
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_51_planner_subplot_2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Subplot 2 -- pie chart showing memory proportions:

In [ ]:
    # Subplot 2: Pie chart — memory proportions
    ax2 = fig.add_subplot(2, 2, 2)
    pie_labels = ['Weights', 'Gradients', 'Optimizer', 'Activations']
    pie_colors = ['#3B82F6', '#10B981', '#F59E0B', '#8B5CF6']
    explode = (0, 0, 0.05, 0.02)

    wedges, texts, autotexts = ax2.pie(
        values, labels=pie_labels, autopct='%1.0f%%', colors=pie_colors,
        explode=explode, startangle=90, textprops={'fontsize': 10}
    )
    ax2.set_title(f'Memory Proportions\n(Total: {mem["total_gb"]:.0f} GB)')

In [ ]:
#@title 🎧 Code Walkthrough: Planner Subplot
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_52_planner_subplot_3.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Subplot 3 -- training time at different GPU counts, with color-coded bars:

In [ ]:
    # Subplot 3: Training time at different GPU counts
    ax3 = fig.add_subplot(2, 2, 3)
    gpu_counts = [1, 8, 64, 256, 1024, 4096, 16384]
    days = [gpu_hours_1 / g / 24 for g in gpu_counts]

    bar_colors = ['#EF4444' if d > 365 else '#F59E0B' if d > 30 else '#10B981' for d in days]
    bars = ax3.bar(range(len(gpu_counts)), days, color=bar_colors, edgecolor='white')
    ax3.set_xlabel(f'Number of {gpu_name} GPUs')
    ax3.set_ylabel('Training Time (days)')
    ax3.set_title(f'Training Time ({num_tokens_trillions}T tokens)')
    ax3.set_xticks(range(len(gpu_counts)))
    ax3.set_xticklabels([f'{g:,}' for g in gpu_counts], rotation=45, fontsize=9)
    ax3.set_yscale('log')
    ax3.yaxis.set_major_formatter(ticker.ScalarFormatter())

    for i, (bar, d) in enumerate(zip(bars, days)):
        if d > 365:
            label = f'{d/365:.0f}y'
        elif d > 1:
            label = f'{d:.0f}d'
        else:
            label = f'{d*24:.0f}h'
        ax3.text(bar.get_x() + bar.get_width() / 2, bar.get_height() * 1.15,
                 label, ha='center', va='bottom', fontsize=8, fontweight='bold')

In [ ]:
#@title 🎧 What to Look For: Planner Subplot Render
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_53_planner_subplot_4_render.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Subplot 4 -- summary text panel with all key numbers, then render the figure:

In [ ]:
# 📊 Visualization
    # Subplot 4: Summary text panel
    ax4 = fig.add_subplot(2, 2, 4)
    ax4.axis('off')
    s = '─' * 40
    summary_text = (
        f"MODEL CONFIGURATION\n{s}\n"
        f"Parameters:        {num_params_billions}B\n"
        f"Hidden dim:        {hidden_dim}\n"
        f"Layers:            {num_layers}\n"
        f"Batch size:        {batch_size}\n"
        f"Sequence length:   {seq_len}\n"
        f"Training tokens:   {num_tokens_trillions}T\n\n"
        f"MEMORY REQUIREMENTS\n{s}\n"
        f"Weights:     {mem['weights_gb']:.1f} GB\n"
        f"Gradients:   {mem['gradients_gb']:.1f} GB\n"
        f"Optimizer:   {mem['optimizer_gb']:.1f} GB\n"
        f"Activations: {mem['activations_gb']:.1f} GB\n"
        f"TOTAL:       {mem['total_gb']:.1f} GB\n\n"
        f"GPU REQUIREMENTS ({gpu_name})\n{s}\n"
        f"GPU memory:  {gpu_memory_gb} GB | Min GPUs: {min_gpus}\n"
        f"FLOPs: {total_flops:.2e} | 1 GPU: {gpu_hours_1/(24*365):.0f} years\n"
    )
    ax4.text(0.05, 0.95, summary_text, transform=ax4.transAxes,
             fontsize=10, verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='#F3F4F6', edgecolor='#D1D5DB'))
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()
    print(f"Your {num_params_billions}B model needs {mem['total_gb']:.0f} GB "
          f"— that is {min_gpus} {gpu_name} GPUs at minimum.")

In [ ]:
#@title 🎧 Narration: Planner 7b Analysis
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_54_planner_7b_analysis.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us use our planner. First, let us analyze a 7B model — the "smallest" modern LLM:

In [ ]:
# 7B model analysis
memory_compute_planner(
    num_params_billions=7,
    batch_size=1,
    seq_len=2048,
    hidden_dim=4096,
    num_layers=32,
    num_tokens_trillions=2.0,
    gpu_name='A100',
    gpu_memory_gb=80,
    gpu_tflops=312,
)

In [ ]:
#@title 🎧 Narration: Planner 405b Analysis
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_55_planner_405b_analysis.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us look at a truly large model — Llama 3 405B:

In [ ]:
# Llama 3 405B analysis
memory_compute_planner(
    num_params_billions=405,
    batch_size=1,
    seq_len=8192,
    hidden_dim=16384,
    num_layers=126,
    num_tokens_trillions=15.0,
    gpu_name='A100',
    gpu_memory_gb=80,
    gpu_tflops=312,
)

print("\nYou now understand exactly why we need parallelism!")
print("A 405B model needs thousands of GPUs — both for memory and compute.")
print("In the next notebooks, we will learn exactly HOW to split the work.")

In [ ]:
#@title 🎧 Listen: Try It Yourself
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_56_try_it_yourself.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Try It Yourself: Custom Model

Modify the parameters below to explore any model configuration you like. Try answering these questions:
- How does doubling the sequence length affect memory?
- What is the memory breakdown for a 1 trillion parameter model?
- How many H100 GPUs would you need for DeepSeek-V3?

In [ ]:
# -- Modify these parameters and re-run! --

memory_compute_planner(
    num_params_billions=70,       # Try: 7, 13, 70, 175, 405, 671, 1000
    batch_size=4,                 # Try: 1, 2, 4, 8, 16
    seq_len=4096,                 # Try: 2048, 4096, 8192, 32768, 131072
    hidden_dim=8192,              # Must match model architecture
    num_layers=80,                # Must match model architecture
    num_tokens_trillions=15.0,    # Training dataset size
    gpu_name='H100',              # Try: 'A100', 'H100'
    gpu_memory_gb=80,             # A100: 80, H100: 80
    gpu_tflops=990,               # A100: 312, H100: 990 (fp16 peak)
)

In [ ]:
#@title 🎧 Narration: Reflection Questions
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_57_reflection_questions.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Reflection and Next Steps

### 🤔 Reflection Questions

Take a moment to think about these questions. They test whether you have truly internalized the material.

**1. Why do optimizer states dominate the memory budget?**

Adam stores three fp32 buffers per parameter: a copy of the weights, the first moment (mean of gradients), and the second moment (mean of squared gradients). Each is 4 bytes, totaling 12 bytes — compared to just 2 bytes for the fp16 weight itself. This means 75% of model state memory is optimizer state. This insight is the foundation of ZeRO, which we will study in the next notebook.

**2. If we doubled the sequence length, which memory component grows the most?**

Activations. The model state (weights, gradients, optimizer) does not depend on sequence length at all — those are fixed by the number of parameters. But activation memory scales with sequence length (and even quadratically for attention scores). Doubling the sequence length roughly doubles (or more than doubles) activation memory.

**3. Why can we not just use CPUs with more memory instead of GPUs?**

CPUs have far less compute throughput. An A100 GPU delivers ~312 TFLOPS of fp16 compute, while a modern CPU manages maybe 1-5 TFLOPS. Even with unlimited memory, a CPU would take 60-300x longer to train the same model. The parallelism problem is fundamentally about both memory *and* compute — you need fast processors with sufficient total memory, and that means many GPUs.

In [ ]:
#@title 🎧 Listen: Optional Challenges
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_58_optional_challenges.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Optional Challenges

If you want to go deeper, try these extensions:

**Challenge 1: Add fp8 Training Support**

Modify the `compute_full_memory` function to support fp8 (1 byte per weight/gradient). How much memory does this save? Note that optimizer states remain in fp32.

**Challenge 2: MoE Memory Accounting**

For a Mixture of Experts model like DeepSeek-V3 (671B total params, 37B active per token, 256 experts), only the active parameters contribute to compute FLOPs, but *all* parameters need memory. Modify the planner to separately report "active" vs "total" parameters and show how this affects the compute vs. memory tradeoff.

**Challenge 3: Gradient Checkpointing**

Gradient checkpointing (also called activation recomputation) trades compute for memory: instead of storing all activations, we only store activations at "checkpoint" layers and recompute the rest during the backward pass. This typically reduces activation memory by $\sqrt{L}$ (where $L$ is the number of layers) at the cost of ~33% more compute. Add this option to your planner.

In [ ]:
#@title 🎧 Before You Start: Challenge Fp8
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_59_challenge_1_fp8.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Challenge 1 starter code — fp8 training support

def compute_memory_fp8(num_params_billions, batch_size=1, seq_len=2048,
                        hidden_dim=4096, num_layers=32):
    """
    Compute memory for fp8 training.

    In fp8 training:
    - Weights: 1 byte per parameter (fp8)
    - Gradients: 1 byte per parameter (fp8)
    - Optimizer states: STILL 12 bytes per parameter (Adam stays in fp32)
    - Activations: 1 byte per element (fp8)
    """
    # ============ TODO ============
    # Implement fp8 memory calculation
    # Compare with fp16 — how much do you save?
    # ==============================
    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Before You Start: Challenge Moe
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_60_challenge_2_moe.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Challenge 2 starter code — MoE memory accounting

def compute_memory_moe(total_params_B, active_params_B, num_experts,
                        top_k, tokens_T, gpu_tflops=312):
    """
    For MoE models, separate memory (all params) from compute (active params).

    DeepSeek-V3: total=671B, active=37B, experts=256, top_k=8
    """
    # ============ TODO ============
    # Memory uses total_params_B (all experts stored)
    # Compute uses active_params_B (only top_k experts active)
    # ==============================
    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_61_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
print("Complete the challenges above to deepen your understanding!")
print()
print("─" * 60)
print("Next up: Notebook 2 — Data Parallelism (\"Hire More Chefs\")")
print("We will implement AllReduce from scratch and understand ZeRO.")
print("─" * 60)